In [1]:
import pandas as pd
import random

In [2]:
!ls /stor/work/Marcotte/project/rmcox/leca/ppi_ml/results/elutions

all_euk_cfms.concat.norm.tidy
all_euk_cfms.concat.raw.tidy
leca_clustering_cmplx_annots.csv
leca_clustering.csv
leca_euk_elut.dollonogs.totalcounts.txt
leca_euks_elut.filt150p.hclust.csv
leca_euks_elut.filt150p.raw.txt
leca_euks_elut.filt300p.hclust.csv
leca_euks_elut.filt300p.hclust.nosums.csv
leca_euks_elut.filt300p.raw.txt
leca_euks_elut.filtdollo.filt150p.hclust.csv
leca_euks_elut.filtdollo.filt150p.hclust.sorted.csv
leca_euks_elut.filtdollo.filt150p.raw.csv
leca_euks_elut.filtdollo.filt150p.raw.noheaders.csv
leca_euks_elut.filtdollo.filt150p.raw.noheaders.txt
leca_euks_elut.filtdollo.filt150p.raw.txt
leca_euks_elut.filtdollo.hclust.csv
leca_euks_elut.filtdollo.norm.csv
leca_euks_elut.filtdollo.norm.noheaders.csv
leca_euks_elut.filtdollo.norm.txt
leca_euks_elut.filtdollo.raw.txt
leca_euks_elut_norm.csv
leca_euks_elut.norm.noheaders.csv
leca_euks_elut.norm.noheaders.txt
leca_euks_elut.norm.txt
leca_euks_elut.raw.txt
leca_vs_all_count_distribution.txt


In [3]:
workdir = '/stor/work/Marcotte/project/rmcox/leca/'
elut_file = workdir+'ppi_ml/results/elutions/leca_euks_elut_norm.csv'
meta_file = workdir+'ppi_ml/data/meta/leca_master_gsheet.csv'

In [4]:
cols = open(elut_file).readline().rstrip()
cols = cols.split(',')
print(cols[0:5])

['orthogroup', 'AT_indark_IEX_fraction_01a_20150422', 'AT_indark_IEX_fraction_02a_20150422', 'AT_indark_IEX_fraction_03a_20150422', 'AT_indark_IEX_fraction_04a_20150422']


In [23]:
fracs = cols[1:]
print(f'# fraction names = {len(fracs)}')
print(f'# unique fraction names = {len(set(fracs))}')

# fraction names = 10481
# unique fraction names = 10481


Need to extract these cols from meta:
- Fractionation Prefix
- Code
- MSBlender Directory
- \# Fractions
- Data Source
- Map

In [6]:
subset = ['Fractionation Prefix', 'Code', 'MSBlender Directory', '# Fractions', 'Data Source', 'Map']
meta = pd.read_csv(meta_file)
df = meta[meta.columns.intersection(subset)]
df.head()

,Fractionation Prefix,Code,MSBlender Directory,# Fractions,Data Source,Map
0,AT_indark_IEX_fraction,ARATH,iex_1,83.0,internal,plant
1,AT_indark_IEX_fraction,ARATH,iex_2,88.0,internal,plant
2,WAN141117_At_Col_0,ARATH,iex_3,77.0,internal,plant
3,AT_leaf_IEX_fraction,ARATH,iex_4,87.0,internal,plant
4,Athaliana_seed_IEX,ARATH,iex_5,60.0,internal,plant


In [7]:
df['Fractionation Prefix']

0      AT_indark_IEX_fraction
1      AT_indark_IEX_fraction
2          WAN141117_At_Col_0
3        AT_leaf_IEX_fraction
4          Athaliana_seed_IEX
                ...          
178                      Xcap
179                  Frog_RNA
180                      Frog
181                     Frog2
182           WAN140125_YEAST
Name: Fractionation Prefix, Length: 183, dtype: object

In [8]:
df.head()

,Fractionation Prefix,Code,MSBlender Directory,# Fractions,Data Source,Map
0,AT_indark_IEX_fraction,ARATH,iex_1,83.0,internal,plant
1,AT_indark_IEX_fraction,ARATH,iex_2,88.0,internal,plant
2,WAN141117_At_Col_0,ARATH,iex_3,77.0,internal,plant
3,AT_leaf_IEX_fraction,ARATH,iex_4,87.0,internal,plant
4,Athaliana_seed_IEX,ARATH,iex_5,60.0,internal,plant


In [9]:
test_fracs = random.sample(fracs, 100)

Create a dictionary mapping common fraction nomenclature to species codes for future sorting into clades. Problematic naming schemes will be stored in `bad_prefix` and corrected manually.

Actually, let's create our problem tuple so we can use the following statement to sort away problem fractions:
`if any(my_str.startswith(item) for item in my_list):`

In [10]:
prefix_dict = dict()
problems = ['WAN']
issues = []
dupes = []
for i in range(len(df)):
    prefix = str(df["Fractionation Prefix"].iloc[i])
    code = df["Code"].iloc[i].lower()
    if prefix.startswith('WAN') and '*' in prefix:
        bad_prefix.append(prefix)
    if prefix.startswith('OT25cm'):
        bad_prefix.append(prefix)
    if prefix.startswith('PT'):
        bad_prefix.append(prefix)
    if prefix not in prefix_dict.keys():
        prefix_dict.update({prefix: code})
    else:
        if code != prefix_dict[prefix]:
            issues.append(prefix)
            prefix_dict.pop(prefix)
        if prefix not in dupes:
            dupes.append(prefix)

In [11]:
print(f'# experiments in meta file = {len(df)}; actual = 178; actual euks = 149')
print(f'# unique prefixes = {len(prefix_dict)}')
print(f'# of prefixes that will map to multiple species = {len(set(bad_prefix))}')

# experiments in meta file = 183; actual = 178; actual euks = 149
# unique prefixes = 171
# of prefixes that will map to multiple species = 34


In [12]:
issues

['nan']

**Problems:**
- OC25cm --> maps to PLABA (berghei), PLAF7 (falciparum), PLAKH (knowlesi)
- PT --> maps to TRYB2 (trypanosome, external), HUMAN (internal)
- WAN10/WAN11/WAN12/WAN13 --> maps to HUMAN, STRPU (sea urchin), CAEEL (c. elegans), MOUSE, NEMVE (sea anemone)

**Solutions:**
- OC25/WAN <-- Take suffix from meta file "Fractionation Prefix" column; search for actual fraction suffix in the meta file suffix; does not fix "PT"
- PT <-- Search meta file suffix in the actual fraction name (whole)
- Update `prefix_dict` accordingly?

In [13]:
set(bad_prefix)

{'OT25cm_CH13_FalC_zero',
 'OT25cm_CH14_Falc_Point',
 'OT25cm_CH15_Falc_One',
 'OT25cm_CH16_KNOW_Zero',
 'OT25cm_CH17_KNOW_Point',
 'OT25cm_CH18_Know_One',
 'OT25cm_CH19_Know_Zero',
 'OT25cm_CH20_berg_zero',
 'OT25cm_CH21_berg_One',
 'OT25cm_CH22_Know_Point',
 'OT25cm_CH23_Know_One',
 'OT25cm_CH24_Berg_Point',
 'OT25cm_CH25_Falc_One',
 'OT25cm_CH26_Falc_Zero',
 'OT25cm_CH27_Falc_Point',
 'PT1541S1',
 'PT4651',
 'PT5528',
 'PTSS4538',
 'WAN100*_HS3NE_HCW',
 'WAN100*_OT2_HS3NE_HCW',
 'WAN101*_OT2_Hela',
 'WAN110*_HCW_HEK293NE',
 'WAN110*_OT2_G166wce_HCW',
 'WAN110*_OT2_mES_HCW',
 'WAN1100*_OT2_CB660_HCW',
 'WAN1100*_OT2_G166wce_HCW',
 'WAN120*_HCW_mES_wce',
 'WAN120*_OT1_HCW_hek293ne',
 'WAN120*_OT2_HCW_SA',
 'WAN121*_OT2_IEX_SA',
 'WAN121*_sea_anemone_IEX',
 'WAN130*_OT2_IEX_SA',
 'WAN130*_sea_anemone_IEX'}

Fraction matching needs to be done in three steps:
1. Find the experiment where the fraction prefix matches the meta file & update 
2. Search for the experiment in the entire fraction name (^PT)
3. Search for the experiment suffix in the entire fraction name (^WAN*, ^OT25cm*)

In [14]:
fraction_dict = dict()
issues = []
dupes = []
nomatch = []
for f in test_fracs:
    fmatch = f.split('_')[0]
    fmatch = fmatch.split('.')[0]
    flag = 0
    for prefix in prefix_dict:
        if prefix.find(fmatch) != -1:
            flag = 1
            if f not in fraction_dict.keys():
                fraction_dict.update({f: prefix_dict[prefix]})
            else:
                if prefix_dict[prefix] != fraction_dict[f]:
                    issues.append(f)
                    fraction_dict.pop(f)
                if f not in dupes:
                    dupes.append(f)
    if flag == 0:
        nomatch.append(f)

In [15]:
print(len(nomatch))
print(len(issues))

problem_children = issues+nomatch
print(len(set(problem_children)))

41
13
46


In [16]:
problem_grandchildren = []
for f in problem_children:
    suffix = f.split('_')
    try:
        smatch = '_'+suffix[1]+'_'+suffix[2]
    except:
        print(f'{suffix} is a problem')
    flag = 0
    for prefix in prefix_dict:
        if smatch in prefix:
            flag = 1
            if f not in fraction_dict.keys():
                fraction_dict.update({f: prefix_dict[prefix]})
            else:
                if prefix_dict[prefix] != fraction_dict[f]:
                    problem_grandchildren.append(f)
                    fraction_dict.pop(f)
                if f not in dupes:
                    dupes.append(f)
    if flag == 0:
        problem_grandchildren.append(f)

['PTSS4538SN19030'] is a problem
['cyto07.a'] is a problem
['PTSS4538SN19050'] is a problem


In [17]:
problem_grandchildren

['P_tricornutum_IEX_18a_03292017',
 'OT25cm_CH26_Falc_Zero.30',
 'OT25cm_CH14_Falc_Point.25',
 'WAN120118_OT1_HCW_hek293ne_P1A03',
 'WAN120726_OT2_HCW_SA_P1C12',
 'WAN120118_OT1_HCW_hek293ne_P1F06',
 'WAN120726_OT2_HCW_SA_P1B06']

In [19]:
#fraction_dict['PTSS4538SN19034']

KeyError: 'PTSS4538SN19034'

In [20]:
dupes

['P_tricornutum_IEX_18a_03292017',
 'Fern_Frond_WWC_50a_20160119',
 'Tetrahymena_body_IEX_27a_06172017',
 'Tetrahymena_cilia_SEC_14a_07072017',
 'RiceL_IEX_29.1a_20150604',
 'OT25cm_CH19_Know_Zero.16',
 'MES_SEC_25_1a_20171201',
 'AT_indark_IEX_fraction_73',
 'RiceL_IEX_82.1a_20150615',
 'AT_leaf_IEX_fraction_80a_20150120',
 'OT25cm_CH26_Falc_Zero.30',
 'OT25cm_CH14_Falc_Point.25',
 'Chlamy_SEC_28a_09132016',
 'OT25cm_CH23_Know_One.42',
 'BroccNuc_IEF_3a_09162016',
 'Rice_leaf_IEX_fraction_89a_20150523',
 'Soy_SEC_DSSO_34a_07232018',
 'Tetr_Diffrac_Control_48a_02242020',
 'Rice_leaf_IEX_fraction_14a_20150507',
 'WAN120118_OT1_HCW_hek293ne_P1A03',
 'WAN120726_OT2_HCW_SA_P1C12',
 'PH091119_HCW090828_293NE_P4B09',
 'PTSS4538SN19030',
 'PH091112_HCW090828_293NE_P3H05',
 'PH090807_HS3NE_HCW_P1B11',
 'PH090716_HS3NE_HCW_P2A04',
 'PH090716_HS3NE_HCW_P2B10',
 'PH090616_HS3CE_TCS_P3D09',
 'WAN120118_OT1_HCW_hek293ne_P1F06',
 'PH090616_HS3CE_TCS_P3D05',
 'WAN120726_OT2_HCW_SA_P1B06',
 'PH090616_

In [21]:
issues

['P_tricornutum_IEX_18a_03292017',
 'P_tricornutum_IEX_18a_03292017',
 'P_tricornutum_IEX_18a_03292017',
 'P_tricornutum_IEX_18a_03292017',
 'P_tricornutum_IEX_18a_03292017',
 'OT25cm_CH19_Know_Zero.16',
 'OT25cm_CH19_Know_Zero.16',
 'OT25cm_CH26_Falc_Zero.30',
 'OT25cm_CH26_Falc_Zero.30',
 'OT25cm_CH14_Falc_Point.25',
 'OT25cm_CH14_Falc_Point.25',
 'OT25cm_CH23_Know_One.42',
 'OT25cm_CH23_Know_One.42']

In [29]:
check_frac = 'P_tricornutum'
print('Checking meta prefix dict ...')
for key in prefix_dict:
    if key.startswith(check_frac):
        print(f'{key}: {prefix_dict[key]}')

print('Checking fraction dict ...')
for key in fraction_dict:
    if key.startswith(check_frac):
        print(f'{key}: {fraction_dict[key]}')
    elif check_frac in key:
        print(f'{key}: {fraction_dict[key]}')

Checking meta prefix dict ...
P_tricornutum_IEX: phatc
P_tricornutum_SEC: phatc
Checking fraction dict ...
P_tricornutum_IEX_18a_03292017: phatc
